In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path= '/content/drive/MyDrive/news dataset/children stories/scraped/'

In [ ]:
os.chdir(path)

In [ ]:
import pandas as pd
df = pd.read_csv('stories.csv')

In [ ]:
df.iloc[9]

title                               Wildfire   |Chapter 3|
story    ['|Chapter 3|', 'Oh no.', 'Oh no, no no!', "Th...
class                                            adventure
url            https://www.bedtime.com/wildfire-chapter-3/
Name: 9, dtype: object

In [ ]:
len(df)

311

In [ ]:
for i in reversed(range(len(df))):
  if df.iloc[i]['title'].find("Chapter") != -1:
    df = df.drop(i, axis=0)

In [ ]:
len(df)

303

In [ ]:
import nltk
import re

def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^\w\s.?,]', '', text.replace('\\n', ' ').replace('xa0', ' ')).lower()
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.iloc[9]

title                                  The Wild West Chase
story    ['“Now what?” asked Ivy as she buried her head...
class                                            adventure
url           https://www.bedtime.com/the-wild-west-chase/
Name: 12, dtype: object

In [ ]:
clean_text(df.iloc[0]['story'])

'zibo is a little zebra. you know what a zebra looks like, dont you? right, it is a white horse with black stripes, or is it a black horse with white stripes?, who cares? today zibo explores the world. neighing happily, he steps through the green fields. animals understand people when they talk. a dog knows what you want if you ask it to give a paw. a horse that pulls the cart knows when it has to stop. but human beings do not know precisely what the cow wants, when it moos. when a cat meows, you may think that it wants to eat, but what did it actually ask? maybe it wants a piece of the chocolate that you are eating? who knows? lets go back to zibo, who walks through the fields. he meets a cow and says hello, animal. the cow looks up and answers good day to you too. who are you and what is your name? i am a cow and my name is brownie. are you useful? surely, i give milk to my calves and to the children. how are you called? i am zibo, say brownie, do you also give milk to the grownups? 

In [ ]:
for i in range(len(df)):
  cleaned=clean_text((df.iloc[i]['story']))
  df.iloc[i]['story']=cleaned


In [ ]:
df.iloc[0]['story']

'zibo is a little zebra. you know what a zebra looks like, dont you? right, it is a white horse with black stripes, or is it a black horse with white stripes?, who cares? today zibo explores the world. neighing happily, he steps through the green fields. animals understand people when they talk. a dog knows what you want if you ask it to give a paw. a horse that pulls the cart knows when it has to stop. but human beings do not know precisely what the cow wants, when it moos. when a cat meows, you may think that it wants to eat, but what did it actually ask? maybe it wants a piece of the chocolate that you are eating? who knows? lets go back to zibo, who walks through the fields. he meets a cow and says hello, animal. the cow looks up and answers good day to you too. who are you and what is your name? i am a cow and my name is brownie. are you useful? surely, i give milk to my calves and to the children. how are you called? i am zibo, say brownie, do you also give milk to the grownups? 

In [ ]:
# text_for_keys=[]
# for text in df['story']:
#   text_for_keys.append(clean_text_for_key(text))

In [ ]:
pip install git+https://github.com/LIAAD/yake

In [ ]:
import yake
# top is changed according to setting of the keywords
kw_extractor = yake.KeywordExtractor(top=10, stopwords=None)

key_yake=[[]]*len(df['story'])
value_yake=[[]]*len(df['story'])
for i in range(len(df['story'])):
  key=[]
  value=[]
  keywords = kw_extractor.extract_keywords(df.iloc[i]['story'])
  for kw, v in keywords:
    key.append(kw)
    value.append(v)
  key_yake[i]=key
  value_yake[i]=value
    

In [ ]:
pip install keybert

In [ ]:
from keybert import KeyBERT
from tqdm import tqdm
kw_model = KeyBERT(model='all-mpnet-base-v2')
key_bert=[[]]*len(df['story'])
value_bert=[[]]*len(df['story'])
for j in tqdm(range(len(df['story']))):
  key=[]
  value=[]
  # top_n is changed according to setting of the keywords
  keywords = kw_model.extract_keywords(df.iloc[j]['story'], keyphrase_ngram_range=(1, 3), stop_words='english', highlight=False,top_n=15)
  for kw, v in keywords:
    key.append(kw)
    value.append(v)
  key_bert[j]=key
  value_bert[j]=value

100%|██████████| 303/303 [02:55<00:00,  1.73it/s]


In [ ]:
key_bert[0]

['animal asks zibo',
 'zibo zebra animal',
 'animal says zibo',
 'zibo little zebra',
 'little zebra know',
 'zebra know',
 'answers animal asks',
 'asks zibo bee',
 'zebra know zebra',
 'zibo millions cows',
 'zibo meets sheep',
 'animals understand',
 'know zebra',
 'sheep think zibo',
 'animal asks']

In [ ]:
#Removing same word detected by both model and keeping only one words
com_keywords=[[]]*len(df)
com_scores=[[]]*len(df)
# Find the common items and delete them
for k in range(len(df)):
  for l in range(len(key_yake[k])-1, -1, -1):  # loop backwards through key_yake
      if key_yake[k][l] in key_bert[k]:
          del key_yake[k][l]
          del value_yake[k][l]
  com_keywords[k]= key_yake[k]+key_bert[k] 
  com_scores[k]=value_yake[k]+value_bert[k]

In [ ]:
com_keywords[0]

['zibo',
 'animal asks zibo',
 'zibo zebra animal',
 'animal says zibo',
 'zibo little zebra',
 'little zebra know',
 'zebra know',
 'answers animal asks',
 'asks zibo bee',
 'zebra know zebra',
 'zibo millions cows',
 'zibo meets sheep',
 'animals understand',
 'know zebra',
 'sheep think zibo',
 'animal asks']

In [ ]:
com_scores[0]

In [ ]:
#appending columns in the dataset
df['keywords']=com_keywords
df['key_scores']=com_scores

In [ ]:
df

,title,story,class,url,keywords,key_scores
0,On the go with Zibo,zibo is a little zebra. you know what a zebra ...,adventure,https://www.bedtime.com/on-the-go-with-zibo/,"[zibo, animal asks zibo, zibo zebra animal, an...","[0.025387348146749516, 0.6537, 0.6, 0.5972, 0...."
1,A Walk in the Woods,he tapped the surface of the pool with the for...,adventure,https://www.bedtime.com/a-walk-in-the-woods/,"[tapped the surface, killed snake, daddy kille...","[0.027205449786734906, 0.5677, 0.5623, 0.5506,..."
2,"A penguin, A black egg and an island","at dawn i heard a knock on the door, it was sa...",adventure,https://www.bedtime.com/a-penguin-a-island-and...,"[leo, friend leo penguin, penguin friend arm, ...","[0.019581578541566145, 0.4863, 0.486, 0.4798, ..."
3,To the Princess,"some years ago, i was a young man and i was on...",adventure,https://www.bedtime.com/to-the-princess/,"[tim, dark lord need, save princess, quest sav...","[0.010967980872248788, 0.4723, 0.4687, 0.4658,..."
4,The Ultimate Quest,"urrggh, owen grunted, he was in the castle doi...",adventure,https://www.bedtime.com/the-ultimate-quest/,"[owen, owen opened book, book ozan owen, confu...","[0.005754948475371684, 0.5256, 0.4947, 0.4797,..."
...,...,...,...,...,...,...
306,Hard Decision,"there was a siberian husky, her name was siber...",warm,https://www.bedtime.com/hard-decision/,"[siberia, siberia friends chihuahua, husky sib...","[0.05583160304862795, 0.5427, 0.5231, 0.5123, ..."
307,HAMILTON THE HAMSTER,by paula moore once upon a time there was a sm...,warm,https://www.bedtime.com/hamilton-the-hamster/,"[hamilton, man realized hamster, realized hams...","[0.01593256329086396, 0.5882, 0.5447, 0.5373, ..."
308,The Lion and a Mouse,one day a mouse was in search of food in the f...,warm,https://www.bedtime.com/the-lion-and-a-mouse/,"[mouse, eaten lion mouse, lion mouse ate, lion...","[0.008304854448105347, 0.6406, 0.6278, 0.6157,..."
309,Friendship,"when is he coming? joseph mumbled, looking dow...",warm,https://www.bedtime.com/friendship/,"[joseph, joseph sit hunting, tree beautiful co...","[0.01597690593545984, 0.4454, 0.4267, 0.4256, ..."


In [ ]:
# saving the DataFrame as a CSV file
df.to_csv('dataset_with_keys_15b.csv', index=False)